SPEED LAYER : S3 + Lambda + Firehose + Elasticsearch Service + Kibana
===
___

**1. Create the bucket that will receive your files**


In [ ]:
!aws s3api create-bucket --bucket $USER-$WORKSHOP-potus-coming --region eu-west-1 \
--create-bucket-configuration LocationConstraint=eu-west-1 

** 2. Create the lambda function used to stream data received on S3 **


- Before creating the lambda function, it is mandatory to create a role for this lambda function. There are 2 steps to assign a role to a specific service : 
    - Create a policy that will allow the service to assume a role
    - Create the policy attached to the role

** 2.a Create the policy that will allow our lambda to assume a role **

In [ ]:
%%bash
echo """{
  \"Version\":\"2012-10-17\",
  \"Statement\": [
    {
      \"Effect\": \"Allow\",
      \"Principal\": {
        \"Service\": \"lambda.amazonaws.com\"
      },
      \"Action\": \"sts:AssumeRole\"
    }
  ]
}">assumerole-lambda-policy.json

** 2.b Create an Inline policy for this lambda function (S3 Get  + S3 List) **

In [ ]:
%%bash
echo "{
  \"Version\": \"2012-10-17\",
  \"Statement\": [
    {
      \"Effect\": \"Allow\",
      \"Action\": [
        \"s3:Get*\",
        \"s3:List*\"
      ],
      \"Resource\": \"arn:aws:s3:::*\"
    }
  ]
}">s3-read-policy.json

** 2.c Create the role associated to the lambda : **

In [ ]:
!aws iam create-role --role-name $USER-$WORKSHOP-role-lambda-speedlayer --assume-role-policy-document file://assumerole-lambda-policy.json

Attached the ** inline ** s3-read-policy to the role

In [ ]:
!aws iam put-role-policy --role-name $USER-$WORKSHOP-role-lambda-speedlayer --policy-name $USER-$WORKSHOP-lambda-s3-r \
  --policy-document file://s3-read-policy.json

Attach 2 **managed** policies to the role : 
- AWS Lambda Basic Execution
- AWS Amazon Kinesis Firehose Full Access

** Lambda Basic Execution **
```javascript
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "logs:CreateLogGroup",
        "logs:CreateLogStream",
        "logs:PutLogEvents"
      ],
      "Resource": "*"
    }
  ]
}```

** Amazon Kinesis Firehose Full Access **
```javascript
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "firehose:*"
            ],
            "Effect": "Allow",
            "Resource": "*"
        }
    ]
}```


In [ ]:
!aws iam attach-role-policy --role-name $USER-$WORKSHOP-role-lambda-speedlayer \
--policy-arn arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole

In [ ]:
!aws iam attach-role-policy --role-name $USER-$WORKSHOP-role-lambda-speedlayer \
--policy-arn arn:aws:iam::aws:policy/AmazonKinesisFirehoseFullAccess

** 2.d Create lambda function with the following parameters **
    - Name : $USER-$WORKSHOP-speedlayer
    - runtime : nodejs6.10
    - role : $USER-$WORKSHOP-role-lambda-speedlayer
    - handler : $USER-$WORKSHOP-function.handler
    - code : S3 source
    - environment : Variable d'env DELIVERY_STREAM_NAME
    - region
    


```javascript
//START
'use strict';

console.log('Loading function');

const aws = require('aws-sdk');
const firehose = new aws.Firehose();
const s3 = new aws.S3({ apiVersion: '2006-03-01' });

function sendToFirehose(recordsForFirehose){
    var paramsFirehose = {
        DeliveryStreamName: process.env.DELIVERY_STREAM_NAME, // required
        Records: recordsForFirehose
    };
    console.log(recordsForFirehose);
    firehose.putRecordBatch(paramsFirehose, function(err, data) {
        if (err) console.log(err, err.stack); // an error occurred
        else     console.log(data);           // successful response
    });
}

function manageData(csvData){
    var recordsForFirehose = new Array();
    var lines = csvData.split("\n"); 
    var i = 0 ;
    var errors = 0 ;
    lines.forEach(function(line) {
        if (line!==""){
            try{
                var fields = line.split("|");
                var isoDate = null ;
                isoDate = new Date(fields[4]).toISOString() ;  
                var recordForFirehose = {
                    id : fields[0],
                    name : fields[1],
                    text : fields[2],
                    date: isoDate,
                    isodate: fields[4],
                    time: fields[3]
                };
                recordsForFirehose[i] = {Data:JSON.stringify(recordForFirehose)} ;
                i++;
            }catch(e){
                errors++;
            }
        }
    });
    console.log("errors:"+errors);
    sendToFirehose(recordsForFirehose)

}

exports.handler = (event, context, callback) => {
    //console.log('Received event:', JSON.stringify(event, null, 2));

    // Get the object from the event and show its content type
    const bucket = event.Records[0].s3.bucket.name;
    const key = decodeURIComponent(event.Records[0].s3.object.key.replace(/\+/g, ' '));
    const params = {
        Bucket: bucket,
        Key: key,
    };
    s3.getObject(params, (err, data) => {
        if (err) {
            console.log(err);
            const message = `Error getting object ${key} from bucket ${bucket}. Make sure they exist and your bucket is in the same region as this function.`;
            console.log(message);
            callback(message);
        } else {
            console.log('CONTENT TYPE:', data.ContentType);
            manageData(data.Body.toString('ascii'));
            callback(null, data.ContentType);
        }
    });
};
//END
```

In [ ]:
!aws lambda create-function --function-name $USER-$WORKSHOP-speedlayer \
--runtime nodejs6.10 --role arn:aws:iam::$ACCOUNTID:role/$USER-$WORKSHOP-role-lambda-speedlayer \
--handler speedlayer-function.handler --code S3Bucket=aws-potus-eu-west-1,S3Key=jupyter/speedlayer-function.zip \
--environment Variables="{DELIVERY_STREAM_NAME=$USER-$WORKSHOP-potus-ds-clinton}" --region eu-west-1

** 2.e Invoke Lambda function when objects are added to s3 **

In [ ]:
!aws lambda add-permission --function-name $USER-$WORKSHOP-speedlayer --region eu-west-1 --statement-id 123456780-$USER-$WORKSHOP \
--action "lambda:InvokeFunction" --principal s3.amazonaws.com \
--source-arn arn:aws:s3:::$USER-$WORKSHOP-potus-coming --source-account $ACCOUNTID

Configure the event that will trigger the lambda function on the bucket S3 

In [ ]:
%%bash
echo "{
    \"LambdaFunctionConfigurations\": [
        {
            \"LambdaFunctionArn\": \"arn:aws:lambda:eu-west-1:$ACCOUNTID:function:$USER-$WORKSHOP-speedlayer\",
            \"Events\": [
                \"s3:ObjectCreated:*\"
            ],
            \"Filter\": {
             \"Key\": {
              \"FilterRules\":[
                {
                 \"Name\": \"prefix\",
                 \"Value\": \"clinton/\"
                }
               ]
              }
             }
        }
    ]
}">s3-trigger-lambda.json

In [ ]:
!aws s3api put-bucket-notification-configuration --bucket $USER-$WORKSHOP-potus-coming \
--notification-configuration file://s3-trigger-lambda.json

The lambda function will now be triggered each time an object is added to the bucket $USER-$WORKSHOP-potus-coming

** 3. ElasticSearch Cluster Creation **


In [ ]:
!aws es create-elasticsearch-domain --domain-name $USER-$WORKSHOP-potus --elasticsearch-version 5.1 \
  --elasticsearch-cluster-config  InstanceType=m3.large.elasticsearch,InstanceCount=1 \
  --ebs-options EBSEnabled=true,VolumeType=gp2,VolumeSize=160 --region eu-west-1

You will need to push a policy to update the access policy in order to be able to access ES and Kibana from your IP.
To get your IP address : https://www.google.fr/search?hl=en&safe=off&q=what+is+my+ip 

***Don't forget to change PUBLICIP in the cell below in the form www.xxx.yyy.zzz.*** 

In [ ]:
%env MY_IP=

In [ ]:
%%bash
echo "{
  \"Version\": \"2012-10-17\",
  \"Statement\": [
    {
      \"Effect\": \"Allow\",
      \"Principal\": \"*\",
      \"Action\": \"es:ESHttp*\",
      \"Resource\": \"arn:aws:es:eu-west-1:$ACCOUNTID:domain/$USER-$WORKSHOP-potus/*\",
      \"Condition\": {
        \"IpAddress\": {
          \"aws:SourceIp\": \"$MY_IP/32\"
        }
      }
    }
  ]
}">es-access-policy.json

In [ ]:
!aws es update-elasticsearch-domain-config --region eu-west-1 \
 --domain-name $USER-$WORKSHOP-potus \
    --access-policies file://es-access-policy.json

You can review you cluster with the following command

In [ ]:
!aws es describe-elasticsearch-domain --domain $USER-$WORKSHOP-potus --region eu-west-1

**4. Create the Kinesis Firehose**

As previously done for the lambda function, before creating a service (here firehose) we need to create a role associated to this service. This is done in 4 steps : 

- Create a policy allowing the firehose to assume a role
- Create a policy for the role (Inline policy)
- Create a role
- Attach the Inline policy to the role
-

In [ ]:
%%bash
echo """{
  \"Version\":\"2012-10-17\",
  \"Statement\": [
    {
      \"Effect\": \"Allow\",
      \"Principal\": {
        \"Service\": \"firehose.amazonaws.com\"
      },
      \"Action\": \"sts:AssumeRole\"
    }
  ]
}">assumerole-firehose-policy.json

In [ ]:
%%bash
echo "{
  \"Version\": \"2012-10-17\",
  \"Statement\": [
    {
      \"Effect\": \"Allow\",
      \"Action\": [
        \"s3:*\",
        \"es:*\"
      ],
      \"Resource\": \"*\"
    }
  ]
}">es-policy.json

In [ ]:
!aws iam create-role --role-name $USER-$WORKSHOP-role-firehose-speedlayer --assume-role-policy-document \
file://assumerole-firehose-policy.json

In [ ]:
!aws iam put-role-policy --role-name $USER-$WORKSHOP-role-firehose-speedlayer --policy-name es-policy \
  --policy-document file://es-policy.json

** Now we create the firehose delivery stream **

In [ ]:
%%bash
echo "{
  \"RoleARN\": \"arn:aws:iam::$ACCOUNTID:role/$USER-$WORKSHOP-role-firehose-speedlayer\",
  \"DomainARN\": \"arn:aws:es:eu-west-1:$ACCOUNT:domain/$USER-$WORKSHOP-potus\",
  \"IndexName\": \"clinton\",
  \"TypeName\": \"message\",
  \"BufferingHints\": {
    \"IntervalInSeconds\": 60,
    \"SizeInMBs\": 5
  },
  \"IndexRotationPeriod\": \"NoRotation\",
  \"RetryOptions\": {
    \"DurationInSeconds\": 300
  },
  \"S3BackupMode\": \"FailedDocumentsOnly\",
  \"S3Configuration\": {
    \"RoleARN\": \"arn:aws:iam::$ACCOUNTID:role/$USER-$WORKSHOP-role-firehose-speedlayer\",
    \"BucketARN\": \"arn:aws:s3:::$USER-$WORKSHOP-potus-coming\"
    }
}">firehose2ES-configuration.json

In [ ]:
!aws firehose create-delivery-stream --delivery-stream-name $USER-$WORKSHOP-potus-ds-clinton \
--delivery-stream-type DirectPut --elasticsearch-destination-configuration file://firehose2ES-configuration.json \
--region eu-west-1


___
***Please wait your Elasticsearch Service Cluster to be ready.*** Configuration state should be **ACTIVE**

To monitor that please look at https://eu-west-1.console.aws.amazon.com/es/home?region=eu-west-1
___

**5a. Now it's time to test our setup with a SAMPLE**


In [ ]:
!aws s3 cp workshop-data-sample/Hillary-450-lines-Sample \
    s3://$USER-$WORKSHOP-potus-coming/clinton/

To follow the result - first wait for 60 seconds :
 - Then go to AWS Lambda console
 - Select your Lambda
 - Select *Monitoring* tab
 - Have a look on *Invocation counter*
 - If you want to have a look on the logs, click the link *View logs in CloudWatch*
 
**5b. Now it's time to send flying data**
 
 We will emulate a live flow by streaming the data corresponding of the day of the election.

In [ ]:
!find workshop-data-sample/last-day-Hillary/ -type f -exec aws s3 cp {} s3://$USER-$WORKSHOP-potus-coming/clinton/ \; \
    -exec sleep 1 \;

**6. KIBANA**

To Finish and to see the result in Kibana 
 - Go to  ES dashboard : https://eu-west-1.console.aws.amazon.com/es/home?region=eu-west-1
 - Click your domain *YOUR-USER-potus*
 - Click the link Kibana
 - Do the initial configuration ; for *Configure an index pattern*
  - Uncheck *Index contains time-based events*
  - For *Index name or pattern* put *clinton*
  - Click *Create*
 - On top click *discover*
  - Click the search icon
 
**7. Usage**
In DISCOVER mode
 - Query : text:*bill*
 - Query : text:*obama*
 - Query : text:*Trump*

**8. Usage**
Creation of a dashboard :
 - create a search called *messages* and save it
 - create 4 vizualisations 
  - one counter to count total messages
  - one counter to count messages containing "debate"
  - one histogram with 20 top user
  - one piechart with top 5 terms
 - creation of the dashboard including these four elements 
 
**DEBUG** - Useful links for debugging: 
 - S3 console : https://console.aws.amazon.com/s3/home?region=eu-west-1#&bucket=YOUR-USER-potus-coming&prefix=
 - Lambda console : https://eu-west-1.console.aws.amazon.com/lambda/home?region=eu-west-1#/
 - Firehose console : https://eu-west-1.console.aws.amazon.com/firehose/home?region=eu-west-1
 - Cloudwatch logs : https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1
 - ES dashboard : https://eu-west-1.console.aws.amazon.com/es/home?region=eu-west-1
 - Kibana with Kibana URL available on ES Dasboard
    

            

___
Time for a DEMO of KIBANA
___